<a href="https://colab.research.google.com/github/calixphd/Quantum-Simulation-Tutorials-and-Examples/blob/main/Genetic_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic algorithms & Combinatorial Optimization

# Product class

In [ ]:
class Product():
  def __init__(self, name, space, price):
    self.name = name
    self.space = space
    self.price = price

In [ ]:
p1 = Product('Refrigerator A', 0.751, 999.9)

In [ ]:
p1.name, p1.space, p1.price

('Refrigerator A', 0.751, 999.9)

In [ ]:
p2 = Product('Cell phone', 0.00000899, 2199.12)

In [ ]:
p2.name, p2.space, p2.price

('Cell phone', 8.99e-06, 2199.12)

In [ ]:
products_list = []
products_list.append(Product('Refrigerator A', 0.751, 999.90))
products_list.append(Product('Cell phone', 0.00000899, 2199.12))
products_list.append(Product('TV 55', 0.400, 4346.99))
products_list.append(Product("TV 50' ", 0.290, 3999.90))
products_list.append(Product("TV 42' ", 0.200, 2999.00))
products_list.append(Product("Notebook A", 0.00350, 2499.90))
products_list.append(Product("Ventilator", 0.496, 199.90))
products_list.append(Product("Microwave A", 0.0424, 308.66))
products_list.append(Product("Microwave B", 0.0544, 429.90))
products_list.append(Product("Microwave C", 0.0319, 299.29))
products_list.append(Product("Refrigerator B", 0.635, 849.00))
products_list.append(Product("Refrigerator C", 0.870, 1199.89))
products_list.append(Product("Notebook B", 0.498, 1999.90))
products_list.append(Product("Notebook C", 0.527, 3999.00)) 

In [ ]:
for product in products_list:
  print(product.name, ' - ', product.price, ' - ', product.space)

Refrigerator A  -  999.9  -  0.751
Cell phone  -  2199.12  -  8.99e-06
TV 55  -  4346.99  -  0.4
TV 50'   -  3999.9  -  0.29
TV 42'   -  2999.0  -  0.2
Notebook A  -  2499.9  -  0.0035
Ventilator  -  199.9  -  0.496
Microwave A  -  308.66  -  0.0424
Microwave B  -  429.9  -  0.0544
Microwave C  -  299.29  -  0.0319
Refrigerator B  -  849.0  -  0.635
Refrigerator C  -  1199.89  -  0.87
Notebook B  -  1999.9  -  0.498
Notebook C  -  3999.0  -  0.527


# Individual class

In [ ]:
from random import random

In [ ]:
random()

0.8719626427907401

In [ ]:
class Individual():
  def __init__(self, spaces, prices, space_limit, generation=0):
    self.spaces = spaces
    self.prices = prices
    self.space_limit = space_limit
    self.score_evaluation = 0
    self.used_space = 0
    self.generation = generation
    self.chromosome = []

    for i in range(len(spaces)):
      if random() < 0.5:
        self.chromosome.append('0')
      else:
        self.chromosome.append('1')

  def fitness(self):
    score = 0
    sum_spaces = 0
    for i in range(len(self.chromosome)):
      if self.chromosome[i] == '1':
        score += self.prices[i]
        sum_spaces += self.spaces[i]
    if sum_spaces > self.space_limit:
      score = 1
    self.score_evaluation = score
    self.used_space = sum_spaces

  def crossover(self, other_individual):
    cutoff = round(random() * len(self.chromosome))
    #print(cutoff)

    child1 = other_individual.chromosome[0:cutoff] + self.chromosome[cutoff::]
    child2 = self.chromosome[0:cutoff] + other_individual.chromosome[cutoff::]
    #print(child1)
    #print(child2)
    children = [Individual(self.spaces, self.prices, self.space_limit, self.generation + 1),
                Individual(self.spaces, self.prices, self.space_limit, self.generation + 1)]
    children[0].chromosome = child1
    children[1].chromosome = child2
    return children

  def mutation(self, rate):
    #print('Before:', self.chromosome)
    for i in range(len(self.chromosome)):
      if random() < rate:
        if self.chromosome[i] == '1':
          self.chromosome[i] = '0'
        else:
          self.chromosome[i] = '1'
    #print('After: ', self.chromosome)
    return self

In [ ]:
0.01

0.01

In [ ]:
random()

0.21006479225262975

# Genectic Algorithm class

In [ ]:
class GeneticAlgorithm():
  def __init__(self, population_size):
    self.population_size = population_size
    self.population = []
    self.generation = 0
    self.best_solution = None
    self.list_of_solutions = []

  def initialize_population(self, spaces, prices, space_limit):
    for i in range(self.population_size):
      self.population.append(Individual(spaces, prices, space_limit))
    self.best_solution = self.population[0]

  def order_population(self):
    self.population = sorted(self.population, key=lambda population: population.score_evaluation, reverse=True)

  def best_individual(self, individual):
    if individual.score_evaluation > self.best_solution.score_evaluation:
      self.best_solution = individual

  def sum_evaluations(self):
    sum = 0
    for individual in self.population:
      sum += individual.score_evaluation
    return sum

  def select_parent(self, sum_evaluation):
    parent = -1
    random_value = random() * sum_evaluation
    sum = 0
    i = 0
    #print('*** random value:', random_value)
    while i < len(self.population) and sum < random_value:
      #print('i:', i, ' - sum: ', sum)
      sum += self.population[i].score_evaluation
      parent += 1
      i += 1
    return parent

  def visualize_generation(self):
    best = self.population[0]
    print('Generation: ', self.population[0].generation,
          'Total price: ', best.score_evaluation, 'Space: ', best.used_space,
          'Chromosome: ', best.chromosome)
    
  def solve(self, mutation_probability, number_of_generations, spaces, prices, limit):
    self.initialize_population(spaces, prices, limit)
    
    for individual in self.population:
      individual.fitness()
    self.order_population()
    self.best_solution = self.population[0]
    self.list_of_solutions.append(self.best_solution.score_evaluation)

    self.visualize_generation()
    
    for generation in range(number_of_generations):
      sum = self.sum_evaluations()
      new_population = []
      for new_individuals in range(0, self.population_size, 2):
        parent1 = self.select_parent(sum)
        parent2 = self.select_parent(sum)
        children = self.population[parent1].crossover(self.population[parent2])
        new_population.append(children[0].mutation(mutation_probability))
        new_population.append(children[1].mutation(mutation_probability))
      
      self.population = list(new_population)

      for individual in self.population:
        individual.fitness()
      self.visualize_generation()
      best = self.population[0]
      self.list_of_solutions.append(best.score_evaluation)
      self.best_individual(best)

    print('**** Best solution - Generation: ', self.best_solution.generation,
          'Total price: ', self.best_solution.score_evaluation, 'Space: ', self.best_solution.used_space,
          'Chromosome: ', self.best_solution.chromosome)
    
    return self.best_solution.chromosome

# Testing the code

In [ ]:
round(random() * 14)

11

In [ ]:
spaces = []
prices = []
names = []
for product in products_list:
  spaces.append(product.space)
  prices.append(product.price)
  names.append(product.name)
limit = 3

In [ ]:
print(spaces)

[0.751, 8.99e-06, 0.4, 0.29, 0.2, 0.0035, 0.496, 0.0424, 0.0544, 0.0319, 0.635, 0.87, 0.498, 0.527]


In [ ]:
print(prices)

[999.9, 2199.12, 4346.99, 3999.9, 2999.0, 2499.9, 199.9, 308.66, 429.9, 299.29, 849.0, 1199.89, 1999.9, 3999.0]


In [ ]:
print(names)

['Refrigerator A', 'Cell phone', 'TV 55', "TV 50' ", "TV 42' ", 'Notebook A', 'Ventilator', 'Microwave A', 'Microwave B', 'Microwave C', 'Refrigerator B', 'Refrigerator C', 'Notebook B', 'Notebook C']


In [ ]:
len(spaces)

14

In [ ]:
names[5], prices[5], spaces[5]

('Notebook A', 2499.9, 0.0035)

In [ ]:
individual1 = Individual(spaces, prices, limit)
#print('Spaces: ', individual1.spaces)
#print('Prices: ', individual1.prices)
#print('Chromosome: ', individual1.chromosome)
for i in range(len(products_list)):
  #print(individual1.chromosome[i])
  if individual1.chromosome[i] == '1':
    print('Name: ', products_list[i].name)
individual1.fitness()
print('Score: ', individual1.score_evaluation)
print('Used space: ', individual1.used_space)
print('Chromosome: ', individual1.chromosome)

Name:  Refrigerator A
Name:  TV 50' 
Name:  TV 42' 
Name:  Notebook A
Name:  Ventilator
Name:  Microwave A
Name:  Microwave C
Name:  Refrigerator B
Name:  Notebook B
Name:  Notebook C
Score:  1
Used space:  3.4748
Chromosome:  ['1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1']


In [ ]:
individual2 = Individual(spaces, prices, limit)
#print('Spaces: ', individual1.spaces)
#print('Prices: ', individual1.prices)
#print('Chromosome: ', individual1.chromosome)
for i in range(len(products_list)):
  #print(individual1.chromosome[i])
  if individual2.chromosome[i] == '1':
    print('Name: ', products_list[i].name)
individual2.fitness()
print('Score: ', individual2.score_evaluation)
print('Used space: ', individual2.used_space)
print('Chromosome: ', individual2.chromosome)

Name:  Cell phone
Name:  TV 55
Name:  Notebook A
Name:  Microwave A
Name:  Microwave B
Name:  Refrigerator B
Name:  Refrigerator C
Name:  Notebook C
Score:  15832.46
Used space:  2.53230899
Chromosome:  ['0', '1', '1', '0', '0', '1', '0', '1', '1', '0', '1', '1', '0', '1']


In [ ]:
children = individual1.crossover(individual2)

In [ ]:
children[0].fitness()
print(children[0].score_evaluation)
print(children[0].chromosome)

16501.86
['0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '1', '0', '1', '1']


In [ ]:
children[1].fitness()
print(children[1].score_evaluation)
print(children[1].chromosome)

1
['1', '0', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1']


In [ ]:
individual1.mutation(0.01)

Before: ['1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1']
After:  ['1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1']


In [ ]:
population_size = 20
ga = GeneticAlgorithm(population_size)
ga.initialize_population(spaces, prices, limit)

In [ ]:
len(ga.population)

20

In [ ]:
ga.population[0].chromosome

['1', '1', '1', '1', '1', '0', '0', '0', '1', '0', '0', '1', '0', '1']

In [ ]:
ga.population[1].chromosome

['0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1']

In [ ]:
for individual in ga.population:
  individual.fitness()
ga.order_population()
for i in range(ga.population_size):
  print('Individual: ', i, '\nSpaces: ', ga.population[i].spaces, '\nPrices: ', ga.population[i].prices,
        '\nChromosome: ', ga.population[i].chromosome, '\nScore: ', ga.population[i].score_evaluation, '\n')

Individual:  0 
Spaces:  [0.751, 8.99e-06, 0.4, 0.29, 0.2, 0.0035, 0.496, 0.0424, 0.0544, 0.0319, 0.635, 0.87, 0.498, 0.527] 
Prices:  [999.9, 2199.12, 4346.99, 3999.9, 2999.0, 2499.9, 199.9, 308.66, 429.9, 299.29, 849.0, 1199.89, 1999.9, 3999.0] 
Chromosome:  ['1', '0', '1', '1', '0', '1', '0', '1', '0', '1', '0', '0', '1', '1'] 
Score:  18453.54 

Individual:  1 
Spaces:  [0.751, 8.99e-06, 0.4, 0.29, 0.2, 0.0035, 0.496, 0.0424, 0.0544, 0.0319, 0.635, 0.87, 0.498, 0.527] 
Prices:  [999.9, 2199.12, 4346.99, 3999.9, 2999.0, 2499.9, 199.9, 308.66, 429.9, 299.29, 849.0, 1199.89, 1999.9, 3999.0] 
Chromosome:  ['0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1'] 
Score:  18203.25 

Individual:  2 
Spaces:  [0.751, 8.99e-06, 0.4, 0.29, 0.2, 0.0035, 0.496, 0.0424, 0.0544, 0.0319, 0.635, 0.87, 0.498, 0.527] 
Prices:  [999.9, 2199.12, 4346.99, 3999.9, 2999.0, 2499.9, 199.9, 308.66, 429.9, 299.29, 849.0, 1199.89, 1999.9, 3999.0] 
Chromosome:  ['1', '0', '1', '1', '0', '1', '0',

In [ ]:
ga.best_solution.score_evaluation

1

In [ ]:
ga.population[0].score_evaluation

18453.54

In [ ]:
ga.best_individual(ga.population[0])

In [ ]:
ga.best_solution.score_evaluation

18453.54

In [ ]:
ga.best_solution.chromosome

['1', '0', '1', '1', '0', '1', '0', '1', '0', '1', '0', '0', '1', '1']

In [ ]:
sum = ga.sum_evaluations()
print('Sum of evaluations: ', sum)

Sum of evaluations:  170661.54


In [ ]:
random() * sum

129546.77230420671

In [ ]:
parent1 = ga.select_parent(sum)
parent1

*** random value: 64166.330798359275
i: 0  - sum:  0
i: 1  - sum:  19504.66
i: 2  - sum:  34980.149999999994
i: 3  - sum:  48113.49999999999
i: 4  - sum:  60708.399999999994


4

In [ ]:
parent2 = ga.select_parent(sum)
parent2

*** random value: 151889.77205909256
i: 0  - sum:  0
i: 1  - sum:  19504.66
i: 2  - sum:  34980.149999999994
i: 3  - sum:  48113.49999999999
i: 4  - sum:  60708.399999999994
i: 5  - sum:  73292.04
i: 6  - sum:  85524.78
i: 7  - sum:  97527.74
i: 8  - sum:  109073.95000000001
i: 9  - sum:  120609.6
i: 10  - sum:  131894.03
i: 11  - sum:  141118.83
i: 12  - sum:  150046.03999999998


12

In [ ]:
new_population = []
mutation_probability = 0.01
for new_individuals in range(0, ga.population_size, 2):
  #print(new_individuals)
  parent1 = ga.select_parent(sum)
  parent2 = ga.select_parent(sum)
  print('\n', parent1, parent2)
  children = ga.population[parent1].crossover(ga.population[parent2])
  print(ga.population[parent1].chromosome)
  print(ga.population[parent2].chromosome)
  print(children[0].chromosome)
  print(children[1].chromosome)

  new_population.append(children[0].mutation(mutation_probability))
  new_population.append(children[1].mutation(mutation_probability))


 3 9
['1', '0', '1', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1']
['1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1']
['1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1']
['1', '0', '1', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1']
Before: ['1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1']
After:  ['1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1']
Before: ['1', '0', '1', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1']
After:  ['1', '1', '1', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1']

 6 1
['0', '0', '0', '1', '1', '1', '0', '1', '0', '0', '0', '1', '1', '0']
['0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1']
['0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '0']
['0', '0', '0', '1', '1', '1', '0', '1', '0', '0', '0', '1', '1', '1']
Before: ['0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '0']
After:  ['0', '0', '1', '

# Putting all together

In [ ]:
products_list = []
products_list.append(Product("Refrigerator A", 0.751, 999.90))
products_list.append(Product("Cell phone", 0.0000899, 2911.12))
products_list.append(Product("TV 55' ", 0.400, 4346.99))
products_list.append(Product("TV 50' ", 0.290, 3999.90))
products_list.append(Product("TV 42' ", 0.200, 2999.00))
products_list.append(Product("Notebook A", 0.00350, 2499.90))
products_list.append(Product("Ventilator", 0.496, 199.90))
products_list.append(Product("Microwave A", 0.0424, 308.66))
products_list.append(Product("Microwave B", 0.0544, 429.90))
products_list.append(Product("Microwave C", 0.0319, 299.29))
products_list.append(Product("Refrigerator B", 0.635, 849.00))
products_list.append(Product("Refrigerator C", 0.870, 1199.89))
products_list.append(Product("Notebook B", 0.498, 1999.90))
products_list.append(Product("Notebook C", 0.527, 3999.00))
spaces = []
prices = []
names = []
for product in products_list:
  spaces.append(product.space)
  prices.append(product.price)
  names.append(product.name)
limit = 3
population_size = 20
mutation_probability = 0.01
number_of_generations = 100
ga = GeneticAlgorithm(population_size)
result = ga.solve(mutation_probability, number_of_generations, spaces, prices, limit)
print(result)
for i in range(len(products_list)):
  if result[i] == '1':
    print('Name: ', products_list[i].name, ' - Price: ', products_list[i].price)

Generation:  0 Total price:  18412.86 Space:  2.3358898999999997 Chromosome:  ['0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '0', '1']
Generation:  1 Total price:  11504.449999999999 Space:  1.8654 Chromosome:  ['0', '0', '1', '1', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0']
Generation:  2 Total price:  1 Space:  3.1093 Chromosome:  ['1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '0', '1', '1', '0']
Generation:  3 Total price:  1 Space:  3.4543 Chromosome:  ['1', '0', '1', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0']
Generation:  4 Total price:  14784.139999999998 Space:  2.8193 Chromosome:  ['1', '0', '1', '0', '1', '1', '0', '1', '1', '0', '0', '1', '1', '0']
Generation:  5 Total price:  14648.46 Space:  2.5412898999999998 Chromosome:  ['1', '1', '0', '1', '0', '1', '0', '1', '1', '1', '0', '1', '1', '0']
Generation:  6 Total price:  16185.19 Space:  2.0833899000000002 Chromosome:  ['0', '1', '1', '0', '1', '0', '0', '0', '1', '1', '0', '1', '0', '1']

In [ ]:
for value in ga.list_of_solutions:
  print(value)

In [ ]:
import plotly.express as px
figure = px.line(x = range(0,101), y = ga.list_of_solutions, title = 'Genetic algorithm results')
figure.show()

# DEAP library - transport of products

- https://github.com/deap/deap

In [ ]:
!pip install deap
#pip install deap == 1.3.1

     |████████████████████████████████| 160 kB 5.1 MB/s 


In [ ]:
class Product():
    def __init__(self, name, space, price):
        self.name = name
        self.space = space
        self.price = price

In [ ]:
products_list = []
products_list.append(Product("Refrigerator A", 0.751, 999.90))
products_list.append(Product("Cell phone", 0.0000899, 2911.12))
products_list.append(Product("TV 55' ", 0.400, 4346.99))
products_list.append(Product("TV 50' ", 0.290, 3999.90))
products_list.append(Product("TV 42' ", 0.200, 2999.00))
products_list.append(Product("Notebook A", 0.00350, 2499.90))
products_list.append(Product("Ventilator", 0.496, 199.90))
products_list.append(Product("Microwave A", 0.0424, 308.66))
products_list.append(Product("Microwave B", 0.0544, 429.90))
products_list.append(Product("Microwave C", 0.0319, 299.29))
products_list.append(Product("Refrigerator B", 0.635, 849.00))
products_list.append(Product("Refrigerator C", 0.870, 1199.89))
products_list.append(Product("Notebook B", 0.498, 1999.90))
products_list.append(Product("Notebook C", 0.527, 3999.00))
spaces = []
prices = []
names = []
for product in products_list:
  spaces.append(product.space)
  prices.append(product.price)
  names.append(product.name)
limit = 3
population_size = 20
mutation_probability = 0.01
number_of_generations = 100 

In [ ]:
import numpy
import random
from deap import base
from deap import creator
from deap import algorithms
from deap import tools

In [ ]:
# [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
def fitness(solution):
  cost = 0
  sum_space = 0
  for i in range(len(solution)):
    if solution[i] == 1:
      cost += prices[i]
      sum_space += spaces[i]
  if sum_space > limit:
    cost = 1
  return cost,

In [ ]:
toolbox = base.Toolbox()
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMax)

In [ ]:
toolbox.register('attr_bool', random.randint, 0, 1)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.attr_bool, n=14)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('evaluate', fitness)
toolbox.register('mate', tools.cxOnePoint)
toolbox.register('mutate', tools.mutFlipBit, indpb = 0.01)
toolbox.register('select', tools.selRoulette)

In [ ]:
population = toolbox.population(n = 20)
crossover_probability = 1.0
number_of_generations = 100

statistics = tools.Statistics(key = lambda individual: individual.fitness.values)
statistics.register('max', numpy.max)
statistics.register('min', numpy.min)
statistics.register('med', numpy.mean)
statistics.register('std', numpy.std)

population, info = algorithms.eaSimple(population, toolbox, crossover_probability, mutation_probability,
                                       number_of_generations, statistics)

gen	nevals	max    	min	med    	std    
0  	20    	16995.5	1  	9871.51	6182.37
1  	20    	18195.3	1  	9956.41	5757.5 
2  	20    	17283.2	1  	10644.3	4672.78
3  	20    	20194.4	7776.69	13039  	3760.04
4  	20    	21255.1	1      	14386.1	5365.76
5  	20    	21255.1	11105.8	16910.9	3191.31
6  	20    	22694.3	10909.1	18033.2	2986.47
7  	20    	22694.3	12756.2	18044.8	2680.06
8  	20    	22255  	14755.3	18900.4	1896.69
9  	20    	22255  	1      	16358.7	7091.34
10 	20    	22385.6	15386.7	19164.9	2045.09
11 	20    	21255.1	1      	17445.7	4595.73
12 	20    	22255  	13275.6	18478.8	2670.15
13 	20    	22255  	15386.7	19785.9	2070.12
14 	20    	22385.6	1      	18115.3	4968.94
15 	20    	23034.7	15486.1	19938.5	2137.81
16 	20    	22684.9	15344  	20143.4	1964.62
17 	20    	23034.7	15344  	19950.4	2496.39
18 	20    	22485  	1      	19255.4	5079.27
19 	20    	22684.9	13774.8	20753.1	2247.7 
20 	20    	22684.9	1      	19435.5	5158.16
21 	20    	22684.9	11345  	20198.3	2595.89
22 	20    	22684.9	15145  	

In [ ]:
best_solutions = tools.selBest(population, 1)
for individual in best_solutions:
  print(individual)
  print(individual.fitness)
  for i in range(len(individual)):
    if individual[i] == 1:
      print('Name: ', names[i], ' - Price: ', prices[i])

[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1]
(22255.000000000004,)
Name:  Refrigerator A  - Price:  999.9
Name:  Cell phone  - Price:  2911.12
Name:  TV 55'   - Price:  4346.99
Name:  TV 50'   - Price:  3999.9
Name:  TV 42'   - Price:  2999.0
Name:  Notebook A  - Price:  2499.9
Name:  Ventilator  - Price:  199.9
Name:  Microwave C  - Price:  299.29
Name:  Notebook C  - Price:  3999.0


In [ ]:
info.select('max')

[16995.47,
 18195.27,
 17283.25,
 20194.37,
 21255.100000000002,
 21255.100000000002,
 22694.270000000004,
 22694.270000000004,
 22255.000000000004,
 22255.000000000004,
 22385.610000000004,
 21255.100000000002,
 22255.000000000004,
 22255.000000000004,
 22385.610000000004,
 23034.710000000003,
 22684.900000000005,
 23034.710000000003,
 22485.000000000004,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22684.900000000005,
 22485.000000000004,
 22485.000000000004,
 22485.000000000004,
 22485.000000000004,
 22485.000000000004,
 22485.000000000004,
 22684.900000000005,
 22485.0000

In [ ]:
import plotly.express as px
figure = px.line(x = range(0,101), y = info.select('max'), title = 'Genetic algorithm results')
figure.show()

# MLROSe library - transport of products

- Documentation: https://mlrose.readthedocs.io/en/stable/source/tutorial1.html#

In [ ]:
!pip install mlrose
#!pip install mlrose == 1.3.0

In [ ]:
import mlrose

In [ ]:
products = [('Refrigerator A', 0.751, 999.90),
            ('Cell phone', 0.0000899, 2911.12),
            ('TV 55', 0.400, 4346.99),
            ('TV 50', 0.290, 3999.90),
            ('TV 42', 0.200, 2999.00),
            ('Notebook A', 0.00350, 2499.90),
            ('Ventilator', 0.496, 199.90),
            ('Microwave A', 0.0424, 308.66),
            ('Microwave B', 0.0544, 429.90),
            ('Microwave C', 0.0319, 299.29),
            ('Refrigerator B', 0.635, 849.00),
            ('Refrigerator C', 0.870, 1199.89),
            ('Notebook B', 0.498, 1999.90),
            ('Notebook C', 0.527, 3999.00)]

In [ ]:
products

[('Refrigerator A', 0.751, 999.9),
 ('Cell phone', 8.99e-05, 2911.12),
 ('TV 55', 0.4, 4346.99),
 ('TV 50', 0.29, 3999.9),
 ('TV 42', 0.2, 2999.0),
 ('Notebook A', 0.0035, 2499.9),
 ('Ventilator', 0.496, 199.9),
 ('Microwave A', 0.0424, 308.66),
 ('Microwave B', 0.0544, 429.9),
 ('Microwave C', 0.0319, 299.29),
 ('Refrigerator B', 0.635, 849.0),
 ('Refrigerator C', 0.87, 1199.89),
 ('Notebook B', 0.498, 1999.9),
 ('Notebook C', 0.527, 3999.0)]

In [ ]:
limit = 3

In [ ]:
def fitness_function(solution):
  cost = 0
  sum_space = 0
  for i in range(len(solution)):
    if solution[i] == 1:
      cost += products[i][2]
      sum_space += products[i][1]
  if sum_space > limit:
    cost = 1
  return cost

In [ ]:
fitness = mlrose.CustomFitness(fitness_function)

In [ ]:
len(products)

14

In [ ]:
problem = mlrose.DiscreteOpt(length=len(products), fitness_fn=fitness, maximize=True, max_val=2) # 0, 1

In [ ]:
best_solution, best_fitness = mlrose.genetic_alg(problem, pop_size=20, mutation_prob=0.01)
best_solution, best_fitness

(array([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1]), 23684.900000000005)

In [ ]:
products[1][0], products[1][2]

('Cell phone', 2911.12)

In [ ]:
for i in range(len(best_solution)):
  if best_solution[i] == 1:
    print('Name: ', products[i][0], ' - Price: ', products[i][2])

Name:  Cell phone  - Price:  2911.12
Name:  TV 55  - Price:  4346.99
Name:  TV 50  - Price:  3999.9
Name:  TV 42  - Price:  2999.0
Name:  Notebook A  - Price:  2499.9
Name:  Ventilator  - Price:  199.9
Name:  Microwave B  - Price:  429.9
Name:  Microwave C  - Price:  299.29
Name:  Notebook B  - Price:  1999.9
Name:  Notebook C  - Price:  3999.0


# Flight schedule - representing the problem

In [ ]:
people = [('Lisbon', 'LIS'),
          ('Madrid', 'MAD'),
          ('Paris', 'CDG'),
          ('Dublin', 'DUB'),
          ('Brussels', 'BRU'),
          ('London', 'LHR')]

In [ ]:
people

[('Lisbon', 'LIS'),
 ('Madrid', 'MAD'),
 ('Paris', 'CDG'),
 ('Dublin', 'DUB'),
 ('Brussels', 'BRU'),
 ('London', 'LHR')]

In [ ]:
people[3]

('Dublin', 'DUB')

In [ ]:
destiny = 'FCO'

In [ ]:
flights = {('BRU', 'FCO'): ['15:44', '18:55', 382]}

In [ ]:
flights

{('BRU', 'FCO'): ['15:44', '18:55', 382]}

In [ ]:
flights[('BRU', 'FCO')]

['15:44', '18:55', 382]

In [ ]:
flights[('BRU', 'FCO')][0], flights[('BRU', 'FCO')][1], flights[('BRU', 'FCO')][2]

('15:44', '18:55', 382)

In [ ]:
flights = {}
for row in open('flights.txt'):
  #print(row)
  #print(row.split(','))
  origin, destiny, departure, arrival, price = row.split(',')
  #print(origin, destiny, departure, arrival, price)
  flights.setdefault((origin, destiny), [])
  #print(flights)
  flights[(origin, destiny)].append((departure, arrival, int(price)))

In [ ]:
flights

{('BRU', 'FCO'): [('6:12', '10:22', 230),
  ('7:53', '11:37', 433),
  ('9:08', '12:12', 364),
  ('10:30', '14:57', 290),
  ('12:19', '15:25', 342),
  ('13:54', '18:02', 294),
  ('15:44', '18:55', 382),
  ('16:52', '20:48', 448),
  ('18:26', '21:29', 464),
  ('20:07', '23:27', 473)],
 ('CDG', 'FCO'): [('6:25', '9:30', 335),
  ('7:34', '9:40', 324),
  ('9:15', '12:29', 225),
  ('11:28', '14:40', 248),
  ('12:05', '15:30', 330),
  ('14:01', '17:24', 338),
  ('15:34', '18:11', 326),
  ('17:07', '20:04', 291),
  ('18:23', '21:35', 134),
  ('19:53', '22:21', 173)],
 ('DUB', 'FCO'): [('6:17', '8:26', 89),
  ('8:04', '10:11', 95),
  ('9:45', '11:50', 172),
  ('11:16', '13:29', 83),
  ('12:34', '15:02', 109),
  ('13:40', '15:37', 138),
  ('15:27', '17:18', 151),
  ('17:11', '18:30', 108),
  ('18:34', '19:36', 136),
  ('20:17', '22:22', 102)],
 ('FCO', 'BRU'): [('6:09', '9:49', 414),
  ('7:57', '11:15', 347),
  ('9:49', '13:51', 229),
  ('10:51', '14:16', 256),
  ('12:20', '16:34', 500),
  ('14:

In [ ]:
flights[('LIS', 'FCO')]

[('6:11', '8:31', 249),
 ('7:39', '10:24', 219),
 ('9:15', '12:03', 99),
 ('11:08', '13:07', 175),
 ('12:18', '14:56', 172),
 ('13:37', '15:08', 250),
 ('15:03', '16:42', 135),
 ('16:51', '19:09', 147),
 ('18:12', '20:17', 242),
 ('20:05', '22:06', 261)]

In [ ]:
flights[('FCO', 'LIS')]

[('6:19', '8:13', 239),
 ('8:04', '10:59', 136),
 ('9:31', '11:43', 210),
 ('11:07', '13:24', 171),
 ('12:31', '14:02', 234),
 ('14:05', '15:47', 226),
 ('15:07', '17:21', 129),
 ('16:35', '18:56', 144),
 ('18:25', '20:34', 205),
 ('20:05', '21:44', 172)]

In [ ]:
flights[('MAD', 'FCO')]

[('6:05', '8:32', 174),
 ('8:25', '10:34', 157),
 ('9:42', '11:32', 169),
 ('11:01', '12:39', 260),
 ('12:44', '14:17', 134),
 ('14:22', '16:32', 126),
 ('15:58', '18:40', 173),
 ('16:43', '19:00', 246),
 ('18:48', '21:45', 246),
 ('19:50', '22:24', 269)]

In [ ]:
flights[('FCO', 'MAD')]

[('6:03', '8:43', 219),
 ('7:50', '10:08', 164),
 ('9:11', '10:42', 172),
 ('10:33', '13:11', 132),
 ('12:08', '14:47', 231),
 ('14:19', '17:09', 190),
 ('15:04', '17:23', 189),
 ('17:06', '20:00', 95),
 ('18:33', '20:22', 143),
 ('19:32', '21:25', 160)]

In [ ]:
schedule = [1,0, 3,2, 7,3, 6,3, 2,4, 5,3]
len(schedule)

12

In [ ]:
len(schedule) // 2

6

In [ ]:
def print_schedule(schedule):
  flight_id = -1
  total_price = 0
  for i in range(len(schedule) // 2):
    name = people[i][0]
    #print(name)
    origin = people[i][1]
    #print(origin)
    flight_id += 1
    going = flights[(origin, destiny)][schedule[flight_id]]
    #print(going)
    total_price += going[2]
    flight_id += 1
    returning = flights[(destiny, origin)][schedule[flight_id]]
    total_price += returning[2]
    #print('\n')
    print('%10s%10s %5s-%5s %3s %5s-%5s %3s' % (name, origin, going[0], going[1], going[2],
                                                returning[0], returning[1], returning[2]))                                                
  print('Total price:', total_price)

In [ ]:
print_schedule(schedule)

    Lisbon       LIS  7:39-10:24 219  6:19- 8:13 239
    Madrid       MAD 11:01-12:39 260  9:11-10:42 172
     Paris       CDG 17:07-20:04 291 11:08-14:38 262
    Dublin       DUB 15:27-17:18 151 10:33-12:03  74
  Brussels       BRU  9:08-12:12 364 12:20-16:34 500
    London       LHR 13:40-15:38 137 10:32-13:16 139
Total price: 2808


In [ ]:
def fitness_function_deap(schedule):
  flight_id = -1
  total_price = 0
  for i in range(0, 6):
    origin = people[i][1]
    flight_id += 1
    going = flights[(origin, destiny)][schedule[flight_id]]
    total_price += going[2]
    flight_id += 1
    returning = flights[(destiny, origin)][schedule[flight_id]]
    total_price += returning[2]
  
  return total_price,

In [ ]:
def fitness_function_mlrose(schedule):
  flight_id = -1
  total_price = 0
  for i in range(0, 6):
    origin = people[i][1]
    flight_id += 1
    going = flights[(origin, destiny)][schedule[flight_id]]
    total_price += going[2]
    flight_id += 1
    returning = flights[(destiny, origin)][schedule[flight_id]]
    total_price += returning[2]
  
  return total_price

# DEAP library - Flight schedule

In [ ]:
toolbox = base.Toolbox()
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
creator.create('Individual', list, fitness=creator.FitnessMin)
toolbox.register('attr_int', random.randint, a = 0, b = 9)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.attr_int, n=12)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('evaluate', fitness_function_deap)
toolbox.register('mate', tools.cxOnePoint)
toolbox.register('mutate', tools.mutFlipBit, indpb = 0.01)
toolbox.register('select', tools.selTournament, tournsize=3)
population = toolbox.population(n = 500)
crossover_probability = 0.7
mutation_probability = 0.3
number_of_generations = 100

statistics = tools.Statistics(key=lambda individuo: individuo.fitness.values)
statistics.register("max", numpy.max)
statistics.register("min", numpy.min)
statistics.register("med", numpy.mean)
statistics.register("std", numpy.std)
    
population, info = algorithms.eaSimple(population, toolbox,
                                       crossover_probability, mutation_probability,
                                       number_of_generations, statistics)

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning:

A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



gen	nevals	max 	min 	med    	std    
0  	500   	3176	1833	2643.69	206.068
1  	391   	2957	1861	2478.49	162.041
2  	399   	2872	1861	2346.94	143.342
3  	409   	2657	1861	2237.3 	132.042
4  	391   	2538	1803	2117.59	118.667
5  	375   	2409	1721	2023.61	104.86 
6  	412   	2280	1696	1943.36	88.1382
7  	385   	2169	1696	1873.37	72.5033
8  	404   	2162	1662	1815.86	63.676 
9  	394   	1954	1622	1764.96	53.0747
10 	406   	1982	1596	1726.16	47.4363
11 	415   	1853	1585	1690.88	37.1378
12 	387   	1919	1579	1665   	40.9808
13 	405   	1843	1573	1635.47	35.4322
14 	395   	1808	1572	1610.48	33.1158
15 	400   	2130	1572	1594.45	38.2075
16 	394   	1786	1572	1583.27	25.0015
17 	391   	1782	1566	1578.46	22.8334
18 	387   	1867	1566	1577.96	28.2307
19 	385   	1833	1566	1576.1 	25.1877
20 	400   	1775	1566	1574.81	24.1348
21 	409   	1767	1566	1570.77	17.4843
22 	379   	1769	1566	1571.82	29.068 
23 	410   	1769	1566	1569.43	22.7524
24 	397   	1769	1566	1570.74	25.6846
25 	381   	1786	1566	1570.21	23.7636
2

In [ ]:
best_solution = tools.selBest(population, 1)
for individual in best_solution:
  print(individual)
  print(individual.fitness)

[2, 6, 5, 7, 8, 1, 3, 6, 0, 2, 9, 1]
(1566.0,)


In [ ]:
print_schedule(individual)

    Lisbon       LIS  9:15-12:03  99 15:07-17:21 129
    Madrid       MAD 14:22-16:32 126 17:06-20:00  95
     Paris       CDG 18:23-21:35 134  8:23-11:07 143
    Dublin       DUB 11:16-13:29  83 15:25-16:58  62
  Brussels       BRU  6:12-10:22 230  9:49-13:51 229
    London       LHR 20:30-23:11 114  8:19-11:16 122
Total price: 1566


# MLROSe library - Flight schedule

In [ ]:
fitness = mlrose.CustomFitness(fitness_function_mlrose)

In [ ]:
problem = mlrose.DiscreteOpt(length=12, fitness_fn=fitness, maximize = False, max_val=10) # 0 - 9

In [ ]:
best_solution, best_fitness = mlrose.genetic_alg(problem, pop_size=500, mutation_prob=0.3)
best_solution, best_fitness

(array([2, 7, 2, 8, 9, 1, 8, 6, 0, 2, 5, 5]), 1807.0)

In [ ]:
print_schedule(best_solution)

    Lisbon       LIS  9:15-12:03  99 16:35-18:56 144
    Madrid       MAD  9:42-11:32 169 18:33-20:22 143
     Paris       CDG 19:53-22:21 173  8:23-11:07 143
    Dublin       DUB 18:34-19:36 136 15:25-16:58  62
  Brussels       BRU  6:12-10:22 230  9:49-13:51 229
    London       LHR 13:40-15:38 137 13:37-15:33 142
Total price: 1807
